In [1]:
import sys, os
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib
#import weatherbench2 as wb
#import metpy
import copy
import wrf
import time
#from metpy.units import units
import cartopy.crs as ccrs
print('modules loaded')
import datetime
import glob
import datetime as dt
import seaborn as sns

modules loaded


In [68]:
year=2020; month=4; day=np.arange(12,14,1); days=day[0]-np.arange(1,10)
#year=2020; month=5; day=23; days=day-np.arange(1,10)
figpath='/users/mfeldman/figs/'
#EVENT SELECTION ERA5
era5='gs://weatherbench2/datasets/era5/1959-2022-6h-1440x721.zarr'
xr_era5=xr.open_zarr(era5)
xr_era5=xr_era5.sel(time=xr_era5.time.dt.year.isin([year]))
xr_era5=xr_era5.sel(time=xr_era5.time.dt.month.isin([month]))#np.arange(3,10)))
xr_era5=xr_era5.sel(time=xr_era5.time.dt.day.isin([day]),latitude=slice(50,20),longitude=slice(250,300))
zsurf_c=xr_era5['geopotential_at_surface']
init=xr_era5.time[0].dt.strftime('%Y%m%d%H').values; print(init)
plevel=copy.deepcopy(xr_era5.level.values)
        
xr_era5['pressure'] = (('level'), plevel)
plevel=xr_era5.pressure
xr_dataset = xr_era5.assign_coords(level=np.arange(len(plevel)))
tlevel=xr_era5.temperature
qlevel=xr_era5.specific_humidity
plevel_dim=np.ones([1,len(plevel),1,1])
zlevel=xr_era5.geopotential
ulevel=xr_era5.u_component_of_wind
vlevel=xr_era5.v_component_of_wind
for n1 in range(len(plevel.data)):
    plevel_dim[0,n1,0,0]=plevel[n1]
plevel_exp=(tlevel/tlevel)*plevel_dim
psurf=xr_era5.mean_sea_level_pressure
tsurf=xr_era5['2m_temperature']
zsurf = zsurf_c.expand_dims(dim={"time": psurf.time}, axis=0)

ps_era=psurf/100
zs=zsurf/9.81
zl=zlevel/9.81

2020041200


In [69]:
era_cape=wrf.cape_2d(pres_hpa=plevel_exp, tkel=tlevel, qv=qlevel, height=zl, terrain=zs, psfc_hpa=ps_era, ter_follow=False)
era_srh=wrf.srhel(u=ulevel, v=vlevel, height=zl, terrain=zs, top=3000.0)

In [70]:
era_cape=era_cape.assign_coords(longitude=psurf.longitude.values);
era_cape=era_cape.assign_coords(latitude=psurf.latitude.values);
era_cape=era_cape.assign_coords(time=psurf.time.values)
cm=plt.cm.Reds
cm.set_bad(color="grey")
cm.set_under(color="grey")
p = era_cape.sel(mcape_mcin_lcl_lfc='mcape',latitude=slice(40,20),longitude=slice(260,290)).plot(x="longitude", y="latitude", col="time", col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()}\
                ,cmap=cm,vmin=200,vmax=3000,levels=np.arange(200,3000,100),extend='both')# col_wrap=4)
for ax in p.axs.flat:
                ax.coastlines()
f=p.fig
f.savefig(figpath+init+'_era5_cape.png',  dpi=300, bbox_inches='tight')
plt.close()

cape_sel= era_cape.sel(mcape_mcin_lcl_lfc='mcape',latitude=slice(40,20),longitude=slice(260,290)).values
cape_sel=cape_sel.reshape(cape_sel.shape[1]*cape_sel.shape[2],cape_sel.shape[0])
cape_sel[cape_sel<200]=np.nan
ax = sns.violinplot(cape_sel,color='grey',density_norm='count',common_norm=True)
ax.set(ylim=(0,3500))
ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)
plt.title('ERA5 CAPE')
plt.ylabel('[J kg$^{-1}$]')
plt.savefig(figpath+init+'_era5_capedist.png',  dpi=300, bbox_inches='tight')
plt.close()


/tmp/ipykernel_3765897/1113614928.py:20: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)


In [63]:
xr_era5.time.dt.strftime('%Y%m%d%H').values

array(['2020041200', '2020041206', '2020041212', '2020041218',
       '2020041300', '2020041306', '2020041312', '2020041318'],
      dtype=object)

In [71]:
era_srh=era_srh.assign_coords(longitude=psurf.longitude.values);
era_srh=era_srh.assign_coords(latitude=psurf.latitude.values);
era_srh=era_srh.assign_coords(time=psurf.time.values)

cm=plt.cm.Reds
cm.set_under(color="grey")
p = era_srh.sel(latitude=slice(50,20),longitude=slice(250,300)).plot(x="longitude", y="latitude", col="time", col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()}\
                ,cmap=cm,vmin=0,vmax=1000,levels=np.arange(0,1000,50),extend='both')# col_wrap=4)
for ax in p.axs.flat:
                ax.coastlines()
f=p.fig
f.savefig(figpath+init+'_era5_srh.png',  dpi=300, bbox_inches='tight')
plt.close()

In [4]:
era_cape=era_cape.assign_coords(longitude=psurf.longitude.values);
era_cape=era_cape.assign_coords(latitude=psurf.latitude.values); era_cape=era_cape.assign_coords(time=psurf.time.values)
era_srh=era_srh.assign_coords(longitude=psurf.longitude.values);
era_srh=era_srh.assign_coords(latitude=psurf.latitude.values); era_srh=era_srh.assign_coords(time=psurf.time.values)

In [73]:
ifs_init='gs://weatherbench2/datasets/hres_t0/2016-2022-6h-1440x721.zarr'
xr_model=xr.open_zarr(ifs_init)
xr_init=xr_model.sel(time=xr_model.time.dt.year.isin([year]))
xr_init=xr_init.sel(time=xr_init.time.dt.month.isin([month]))#np.arange(3,10)))
xr_init=xr_init.sel(time=xr_init.time.dt.day.isin([day]),latitude=slice(20,50),longitude=slice(250,300))
init=xr_init.time[0].dt.strftime('%Y%m%d%H').values; print(init)
#xr_init=xr_init.sel(prediction_timedelta=xr_init.prediction_timedelta[0])
print(xr_init.dims)
#xr_init=xr_init.squeeze(dim='prediction_timedelta')

xr_init = xr_init.sortby('level', ascending=False)
plevel=copy.deepcopy(xr_init.level.values)
xr_init['pressure'] = (('level'), plevel)
plevel=xr_init.pressure
xr_init = xr_init.assign_coords(level=np.arange(len(plevel)))

tlevel=xr_init.temperature
qlevel=xr_init.specific_humidity
plevel_dim=np.ones([1,len(plevel),1,1])
zlevel=xr_init.geopotential
ulevel=xr_init.u_component_of_wind
vlevel=xr_init.v_component_of_wind
for n1 in range(len(plevel.data)):
    plevel_dim[0,n1,0,0]=plevel[n1]
plevel_exp=(tlevel/tlevel)*plevel_dim

psurf=xr_init.mean_sea_level_pressure
tsurf=xr_init['2m_temperature']
zsurf = zsurf_c.expand_dims(dim={"time": psurf.time}, axis=0)

ps=psurf/100
zs=zsurf/9.81
zl=zlevel/9.81

init_cape=wrf.cape_2d(pres_hpa=plevel_exp, tkel=tlevel, qv=qlevel, height=zl, terrain=zs, psfc_hpa=ps, ter_follow=False)
init_srh=wrf.srhel(u=ulevel, v=vlevel, height=zl, terrain=zs, top=3000.0)

init_cape=init_cape.assign_coords(longitude=psurf.longitude.values);
init_cape=init_cape.assign_coords(latitude=psurf.latitude.values);
init_cape=init_cape.assign_coords(time=psurf.time.values)
init_srh=init_srh.assign_coords(longitude=psurf.longitude.values);
init_srh=init_srh.assign_coords(latitude=psurf.latitude.values);
init_srh=init_srh.assign_coords(time=psurf.time.values)

2020041200
FrozenMappingWarningOnValuesAccess({'time': 8, 'latitude': 121, 'longitude': 201, 'level': 13})


In [83]:
cm=plt.cm.Reds
cm.set_bad(color="grey")
cm.set_under(color="grey")
p = init_cape.sel(mcape_mcin_lcl_lfc='mcape',latitude=slice(20,50),longitude=slice(250,300)).plot(x="longitude", y="latitude", col="time", col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()}\
                ,cmap=cm,vmin=200,vmax=3000,levels=np.arange(200,3000,100),extend='both')# col_wrap=4)
for ax in p.axs.flat:
                ax.coastlines()
f=p.fig
f.savefig(figpath+init+'_ifsinit_cape.png',  dpi=300, bbox_inches='tight')
plt.close()

cape_sel= init_cape.sel(mcape_mcin_lcl_lfc='mcape',latitude=slice(20,40),longitude=slice(260,290)).values
cape_sel=cape_sel.reshape(cape_sel.shape[1]*cape_sel.shape[2],cape_sel.shape[0])
cape_sel[cape_sel<200]=np.nan
ax = sns.violinplot(cape_sel,color='grey',density_norm='count',common_norm=True)
ax.set(ylim=(0,3500))
ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)
plt.title('IFS INIT CAPE')
plt.ylabel('[J kg$^{-1}$]')
plt.savefig(figpath+init+'_ifsinit_capedist.png',  dpi=300, bbox_inches='tight')
plt.close()


cm=plt.cm.Reds
cm.set_under(color="grey")
p = init_srh.sel(latitude=slice(20,50),longitude=slice(250,300)).plot(x="longitude", y="latitude", col="time", col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()}\
                ,cmap=cm,vmin=0,vmax=1000,levels=np.arange(0,1000,50),extend='both')# col_wrap=4)
for ax in p.axs.flat:
                ax.coastlines()
f=p.fig
f.savefig(figpath+init+'_ifsinit_srh.png',  dpi=300, bbox_inches='tight')
plt.close()

/tmp/ipykernel_3765897/2886384045.py:17: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)


In [16]:
print(init_cape.shape,init_cape.dims)
print(init_cape.time)

(4, 4, 721, 1440) ('mcape_mcin_lcl_lfc', 'time', 'latitude', 'longitude')
<xarray.DataArray 'time' (time: 4)>
array(['2020-04-12T00:00:00.000000000', '2020-04-12T12:00:00.000000000',
       '2020-04-13T00:00:00.000000000', '2020-04-13T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-04-12 ... 2020-04-13T12:00:00


In [76]:
start=xr_era5.time[0]
end=xr_era5.time[-1]
ref_cape=era_cape; ref_srh=era_srh
ref_cape=init_cape; ref_srh=init_srh

#LOADING FORECAST DATASET
model='pangu'
model='ifs'
# model='fuxi'
# model='keisler'
# model='spherical'
latslice=slice(50,20)
#latslice=slice(20,50)
dataset='gs://weatherbench2/datasets/pangu/2018-2022_0012_0p25.zarr'
#dataset='gs://weatherbench2/datasets/hres/2016-2022-0012-1440x721.zarr'
# dataset='gs://weatherbench2/datasets/fuxi/2020-1440x721.zarr'
# dataset='gs://weatherbench2/datasets/keisler/2020-360x181.zarr'
# dataset='gs://weatherbench2/datasets/sphericalcnn/2020-240x121_equiangular_with_poles.zarr'
# days=[6,7,8,9,10,11,12]
hours=[0,12]
for d in days[:]:
    for hour in hours[:]:
        xr_model=xr.open_zarr(dataset)
        xr_dataset=xr_model.sel(time=xr_model.time.dt.year.isin([year]))
        xr_dataset=xr_dataset.sel(time=xr_dataset.time.dt.month.isin([month]))
        xr_dataset=xr_dataset.sel(time=xr_dataset.time.dt.day.isin([d]))
        xr_dataset=xr_dataset.sel(time=xr_dataset.time.dt.hour.isin([hour]),latitude=latslice,longitude=slice(250,300))
        fcst_init=xr_dataset.time[0]
        init=fcst_init.dt.strftime('%Y%m%d%H').values; print(init)
        xr_dataset=xr_dataset.sel(prediction_timedelta=slice(start-fcst_init,end-fcst_init))
        xr_dataset = xr_dataset.sortby('level', ascending=False)
        
        plevel=copy.deepcopy(xr_dataset.level.values)
        
        xr_dataset['pressure'] = (('level'), plevel)
        plevel=xr_dataset.pressure
        xr_dataset = xr_dataset.assign_coords(level=np.arange(len(plevel)))
        
        tlevel=xr_dataset.temperature
        qlevel=xr_dataset.specific_humidity
        plevel_dim=np.ones([1,len(plevel),1,1])
        zlevel=xr_dataset.geopotential
        ulevel=xr_dataset.u_component_of_wind
        vlevel=xr_dataset.v_component_of_wind
        for n1 in range(len(plevel.data)):
            plevel_dim[0,n1,0,0]=plevel[n1]
        plevel_exp=(tlevel/tlevel)*plevel_dim
        
        psurf=xr_dataset.mean_sea_level_pressure
        tsurf=xr_dataset['2m_temperature']
        zsurf = zsurf_c.expand_dims(dim={"time": psurf.time}, axis=0)
        zsurf = zsurf.expand_dims(dim={"prediction_timedelta": psurf.prediction_timedelta}, axis=1)
        
        ps=psurf/100
        zs=zsurf/9.81
        zl=zlevel/9.81

        cape=wrf.cape_2d(pres_hpa=plevel_exp, tkel=tlevel, qv=qlevel, height=zl, terrain=zs, psfc_hpa=ps, ter_follow=False)
        srh=wrf.srhel(u=ulevel, v=vlevel, height=zl, terrain=zs, top=3000.0)

        cape=cape.assign_coords(longitude=psurf.longitude.values);
        cape=cape.assign_coords(latitude=psurf.latitude.values);
        cape=cape.assign_coords(prediction_timedelta=ps_era.time.values)

        srh=srh.assign_coords(longitude=psurf.longitude.values);
        srh=srh.assign_coords(latitude=psurf.latitude.values);
        srh=srh.assign_coords(prediction_timedelta=ps_era.time.values)

        cape=cape.squeeze(dim='time'); cape=cape.rename({'prediction_timedelta':'time'})
        srh=srh.squeeze(dim='time'); srh=srh.rename({'prediction_timedelta':'time'})
        
        cm=plt.cm.Reds
        cm.set_bad(color="grey")
        cm.set_under(color="grey")
        p = cape.sel(mcape_mcin_lcl_lfc='mcape',latitude=latslice,longitude=slice(250,300)).plot(x="longitude", y="latitude", col="time",\
                         col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},\
                         cmap=cm,vmin=200,vmax=3000,levels=np.arange(200,3000,100),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+init+'_cape.png',  dpi=300, bbox_inches='tight')
        plt.close()

        cape_sel= cape.sel(mcape_mcin_lcl_lfc='mcape',latitude=slice(40,20),longitude=slice(260,290)).values
        cape_sel=cape_sel.reshape(cape_sel.shape[1]*cape_sel.shape[2],cape_sel.shape[0])
        cape_sel[cape_sel<200]=np.nan
        ax = sns.violinplot(cape_sel,color='grey',density_norm='count',common_norm=True)
        ax.set(ylim=(0,3500))
        ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)
        plt.title(model+' CAPE '+ init)
        plt.ylabel('[J kg$^{-1}$]')
        plt.savefig(figpath+model+init+'_capedist.png',  dpi=300, bbox_inches='tight')
        plt.close()

        
        cm=plt.cm.Reds
        cm.set_bad(color="grey")
        cm.set_under(color="grey")
        p = srh.sel(latitude=latslice,longitude=slice(250,300)).plot(x="longitude", y="latitude", col="time",\
                         col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},\
                         cmap=cm,vmin=0,vmax=1000,levels=np.arange(0,1000,50),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+init+'_srh.png',  dpi=300, bbox_inches='tight')
        plt.close()

        #cape=cape.sel(time=cape.time.dt.hour.isin([0,12]))
        #srh=srh.sel(time=srh.time.dt.hour.isin([0,12]))
        cm=plt.cm.RdBu_r
        cm.set_bad(color="grey")
        dcape=cape.fillna(0)-ref_cape.fillna(0)
        dsrh=srh.fillna(0)-ref_srh.fillna(0)
        p = dcape.sel(mcape_mcin_lcl_lfc='mcape',latitude=latslice,longitude=slice(250,300)).plot(x="longitude", y="latitude", col="time",\
                         col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},\
                         cmap=cm,vmin=-1000,vmax=1000,levels=np.arange(-1000,1010,100),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+init+'_cape_d.png',  dpi=300, bbox_inches='tight')
        plt.close()


        cm=plt.cm.RdBu_r
        cm.set_bad(color="grey")
        p = dsrh.sel(latitude=latslice,longitude=slice(250,300)).plot(x="longitude", y="latitude", col="time",\
                         col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},\
                         cmap=cm,vmin=-500,vmax=500,levels=np.arange(-500,505,50),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+init+'_srh_d.png',  dpi=300, bbox_inches='tight')
        plt.close()

2020041100


/tmp/ipykernel_3765897/1249926483.py:89: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)


2020041112


/tmp/ipykernel_3765897/1249926483.py:89: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)


2020041000


...

/tmp/ipykernel_3765897/1249926483.py:89: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)


2020040412


/tmp/ipykernel_3765897/1249926483.py:89: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)


2020040300


ValueError: conflicting sizes for dimension 'prediction_timedelta': length 5 on <this-array> and length 8 on {'mcape_mcin_lcl_lfc': 'mcape_mcin_lcl_lfc', 'longitude': 'longitude', 'latitude': 'latitude', 'prediction_timedelta': 'prediction_timedelta', 'time': <this-array>}

In [80]:
start=xr_era5.time[0]
end=xr_era5.time[-1]
mlpath='/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/'
#mlpath='/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ML_PREDICT/panguweather/Data_Monika/'
mname='graphcast'
#ref_cape=init_cape; ref_srh=init_srh
#LOADING FORECAST DATASET
model='graphcast_dawn'
latslice=slice(50,20)
latslice=slice(20,50)
year=2020; month=4
days=[7,8,9,10,11,12]
hours=[0,12]
#ref_cape=init_cape; ref_srh=init_srh
ref_cape=era_cape; ref_srh=era_srh
for day in days[:]:
    for hour in hours[:]:
        filelist=sorted(glob.glob(mlpath+'*'+mname+'*'+str(year)+str(month).zfill(2)+str(day).zfill(2)+'_t_'+str(hour)+'*'))
        #print(filelist)
        st=datetime.datetime(start.dt.year.values,start.dt.month.values,start.dt.day.values,start.dt.hour.values)
        et=datetime.datetime(end.dt.year.values,end.dt.month.values,end.dt.day.values,end.dt.hour.values)
        dat=datetime.datetime(year, month, day, hour)
        init=dat.strftime('%Y%m%d%H')
        lt1=int((st-dat).total_seconds() / 3600)#,tzinfo=datetime.timezone.utc)
        lt2=int((et-dat).total_seconds() / 3600)
        lt=np.arange(lt1,lt2+1,6).astype(str)
        print(lt)
        myfile=[file for file in filelist if lt[0]+'h.grib' in file and '.idx' not in file][0]#any(part in file for part in lt)]
        print(myfile)
        xr_model=xr.open_dataset(myfile)
        xr_model2=xr.open_dataset(myfile, backend_kwargs={'filter_by_keys': {'shortName': '2t'}})
        times=pd.date_range(st,et,freq='6h')
        for n1 in range(1,len(lt)):
            myfile=[file for file in filelist if lt[n1]+'h.grib' in file and '.idx' not in file][0]#any(part in file for part in lt)]
            print(myfile)
            xr_model=xr.concat([xr_model, xr.open_dataset(myfile)], "time")
            xr_model2=xr.concat([xr_model2, xr.open_dataset(myfile, backend_kwargs={'filter_by_keys': {'shortName': '2t'}})], "time")
        xr_model['time']=times
        xr_model2['time']=times
        xr_model=xr_model.assign(t2m=xr_model2.t2m)
        xr_model=xr_model.sel(latitude=latslice,longitude=slice(250,300))

        plevel=xr_model.isobaricInhPa.values
        tlevel=xr_model.t
        qlevel=xr_model.q
        plevel_dim=np.ones([1,len(plevel),1,1])
        zlevel=xr_model.z
        ulevel=xr_model.u
        vlevel=xr_model.v
        for n1 in range(len(plevel.data)):
            plevel_dim[0,n1,0,0]=plevel[n1]
        plevel_exp=(tlevel/tlevel)*plevel_dim
        
        psurf=xr_model.msl
        tsurf=xr_model.t2m
        zsurf = zsurf_c.expand_dims(dim={"time": psurf.time}, axis=0)
        #zsurf = zsurf.expand_dims(dim={"prediction_timedelta": psurf.prediction_timedelta}, axis=1)
        
        ps=psurf/100
        zs=zsurf/9.81
        zl=zlevel/9.81

        cape=wrf.cape_2d(pres_hpa=plevel_exp, tkel=tlevel, qv=qlevel, height=zl, terrain=zs, psfc_hpa=ps, ter_follow=False)
        srh=wrf.srhel(u=ulevel, v=vlevel, height=zl, terrain=zs, top=3000.0)

        cape=cape.assign_coords(longitude=psurf.longitude.values);
        cape=cape.assign_coords(latitude=psurf.latitude.values);
        cape=cape.assign_coords(time=psurf.time.values)

        srh=srh.assign_coords(longitude=psurf.longitude.values);
        srh=srh.assign_coords(latitude=psurf.latitude.values);
        srh=srh.assign_coords(time=psurf.time.values)
        
        cm=plt.cm.Reds
        cm.set_bad(color="grey")
        cm.set_under(color="grey")
        p = cape.sel(mcape_mcin_lcl_lfc='mcape',latitude=latslice,longitude=slice(250,300)).plot(x="longitude", y="latitude", col="time",\
                         col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},\
                         cmap=cm,vmin=200,vmax=3000,levels=np.arange(200,3000,100),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+init+'_cape.png',  dpi=300, bbox_inches='tight')
        plt.close()

        cape_sel= cape.sel(mcape_mcin_lcl_lfc='mcape',latitude=slice(20,40),longitude=slice(260,290)).values
        cape_sel=cape_sel.reshape(cape_sel.shape[1]*cape_sel.shape[2],cape_sel.shape[0])
        cape_sel[cape_sel<200]=np.nan
        ax = sns.violinplot(cape_sel,color='grey',density_norm='count',common_norm=True)
        ax.set(ylim=(0,3500))
        ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)
        plt.title(model+' CAPE '+ init)
        plt.ylabel('[J kg$^{-1}$]')
        plt.savefig(figpath+model+init+'_capedist.png',  dpi=300, bbox_inches='tight')
        plt.close()

        
        cm=plt.cm.Reds
        cm.set_bad(color="grey")
        cm.set_under(color="grey")
        p = srh.sel(latitude=latslice,longitude=slice(250,300)).plot(x="longitude", y="latitude", col="time",\
                         col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},\
                         cmap=cm,vmin=0,vmax=1000,levels=np.arange(0,1000,50),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+init+'_srh.png',  dpi=300, bbox_inches='tight')
        plt.close()

        #cape=cape.sel(time=cape.time.dt.hour.isin([0,12]))
        #srh=srh.sel(time=srh.time.dt.hour.isin([0,12]))
        cm=plt.cm.RdBu_r
        cm.set_bad(color="grey")
        dcape=cape.fillna(0)-ref_cape.fillna(0)
        dsrh=srh.fillna(0)-ref_srh.fillna(0)
        p = dcape.sel(mcape_mcin_lcl_lfc='mcape',latitude=latslice,longitude=slice(250,300)).plot(x="longitude", y="latitude", col="time",\
                         col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},\
                         cmap=cm,vmin=-1000,vmax=1000,levels=np.arange(-1000,1010,100),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+init+'_cape_d.png',  dpi=300, bbox_inches='tight')
        plt.close()


        cm=plt.cm.RdBu_r
        cm.set_bad(color="grey")
        p = dsrh.sel(latitude=latslice,longitude=slice(250,300)).plot(x="longitude", y="latitude", col="time",\
                         col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},\
                         cmap=cm,vmin=-500,vmax=500,levels=np.arange(-500,505,50),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+init+'_srh_d.png',  dpi=300, bbox_inches='tight')
        plt.close()

['120' '126' '132' '138' '144' '150' '156' '162']
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/graphcast_d_20200407_t_0000_120h.grib


skipping variable: paramId==165 shortName='u10'
Traceback (most recent call last):
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/cfgrib/dataset.py", line 680, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/cfgrib/dataset.py", line 611, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='heightAboveGround' value=Variable(dimensions=(), data=2.0) new_value=Variable(dimensions=(), data=10.0)
skipping variable: paramId==166 shortName='v10'
Traceback (most recent call last):
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/cfgrib/dataset.py", line 680, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib

/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/graphcast_d_20200407_t_0000_126h.grib


skipping variable: paramId==165 shortName='u10'
Traceback (most recent call last):
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/cfgrib/dataset.py", line 680, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/cfgrib/dataset.py", line 611, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='heightAboveGround' value=Variable(dimensions=(), data=2.0) new_value=Variable(dimensions=(), data=10.0)
skipping variable: paramId==166 shortName='v10'
Traceback (most recent call last):
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/cfgrib/dataset.py", line 680, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib

/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/graphcast_d_20200407_t_0000_132h.grib


...

skipping variable: paramId==165 shortName='u10'
Traceback (most recent call last):
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/cfgrib/dataset.py", line 680, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/cfgrib/dataset.py", line 611, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='heightAboveGround' value=Variable(dimensions=(), data=2.0) new_value=Variable(dimensions=(), data=10.0)
skipping variable: paramId==166 shortName='v10'
Traceback (most recent call last):
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/cfgrib/dataset.py", line 680, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib

/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/graphcast_d_20200412_t_0000_42h.grib


skipping variable: paramId==165 shortName='u10'
Traceback (most recent call last):
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/cfgrib/dataset.py", line 680, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/cfgrib/dataset.py", line 611, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='heightAboveGround' value=Variable(dimensions=(), data=2.0) new_value=Variable(dimensions=(), data=10.0)
skipping variable: paramId==166 shortName='v10'
Traceback (most recent call last):
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/cfgrib/dataset.py", line 680, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib

['-12' '-6' '0' '6' '12' '18' '24' '30']


IndexError: list index out of range

In [81]:
import netCDF4 as nc
import metpy
from metpy.units import units
start=xr_era5.time[0]
end=xr_era5.time[-1]
mlpath='/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/'
#mlpath='/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ML_PREDICT/panguweather/Data_Monika/'
mname='fourcastnetv2'
#ref_cape=init_cape; ref_srh=init_srh
#LOADING FORECAST DATASET
model='fcnv2_dawn'
latslice=slice(50,20)
latslice2=slice(20,50)
year=2020; month=4
days=[7,8,9,10,11,12]
hours=[0,12]
#ref_cape=init_cape; ref_srh=init_srh
ref_cape=era_cape.sel(latitude=latslice,longitude=slice(250,300)); ref_srh=era_srh.sel(latitude=latslice,longitude=slice(250,300))
for day in days[:]:
    for hour in hours[:]:
        myfile=sorted(glob.glob(mlpath+'*'+mname+'*'+str(year)+'-'+str(month).zfill(2)+'-'+str(day).zfill(2)+'T'+str(hour).zfill(2)+'*'))[-1]
        #print(filelist)
        st=datetime.datetime(start.dt.year.values,start.dt.month.values,start.dt.day.values,start.dt.hour.values)
        et=datetime.datetime(end.dt.year.values,end.dt.month.values,end.dt.day.values,end.dt.hour.values)
        dat=datetime.datetime(year, month, day, hour)
        init=dat.strftime('%Y%m%d%H')
        lt1=int((st-dat).total_seconds() / 3600)#,tzinfo=datetime.timezone.utc)
        lt2=int((et-dat).total_seconds() / 3600)
        lt=np.arange(lt1,lt2+1,6).astype(str)
        print(lt)
        # myfile=[file for file in filelist if lt[0]+'_6.nc' in file][0]#any(part in file for part in lt)]
        print(myfile)
        xr_model=xr.open_dataset(myfile)
        xr_model=xr_model.rename({'lat':'latitude'})
        xr_model=xr_model.rename({'lon':'longitude'})
        xr_model=xr_model.squeeze(dim='history')
        xr_model=xr_model.sel(time=slice(start,end))
        xr_model=xr_model.sel(latitude=latslice,longitude=slice(250,300))
        plevel=[50,100,150,200,250,300,400,500,600,700,850,925,1000]
        ulevel = xr_model['__xarray_dataarray_variable__'][:,8:21,:]
        vlevel = xr_model['__xarray_dataarray_variable__'][:,21:34,:]
        zlevel = xr_model['__xarray_dataarray_variable__'][:,34:47,:]
        tlevel = xr_model['__xarray_dataarray_variable__'][:,47:60,:]
        rlevel = xr_model['__xarray_dataarray_variable__'][:,60:73,:]
        psurf = xr_model['__xarray_dataarray_variable__'][:,5,:]
        tsurf = xr_model['__xarray_dataarray_variable__'][:,4,:]

        plevel_dim=np.ones([1,len(plevel),1,1])
        for n1 in range(len(plevel)):
            plevel_dim[0,n1,0,0]=plevel[n1]
        plevel_exp=(tlevel/tlevel)*plevel_dim
        
        dplevel=metpy.calc.dewpoint_from_relative_humidity(tlevel.values* units('K'), rlevel.values* units.percent)
        qlevel=metpy.calc.specific_humidity_from_dewpoint(plevel_exp.values* units('hPa'), dplevel).magnitude
        #.to(units('kg/kg')#.magnitude

        zsurf = zsurf_c.expand_dims(dim={"time": psurf.time}, axis=0)
        zsurf=zsurf.sel(latitude=latslice,longitude=slice(250,300))
        #zsurf = zsurf_c.expand_dims(dim={"history": psurf.history}, axis=1)
        #zsurf = zsurf.expand_dims(dim={"prediction_timedelta": psurf.prediction_timedelta}, axis=1)
        
        ps=psurf/100
        zs=zsurf/9.81
        zl=zlevel/9.81

        cape=wrf.cape_2d(pres_hpa=plevel_exp, tkel=tlevel, qv=qlevel, height=zl, terrain=zs, psfc_hpa=ps, ter_follow=False)
        #srh=wrf.srhel(u=ulevel, v=vlevel, height=zl, terrain=zs, top=3000.0)

        cape=cape.assign_coords(longitude=psurf.longitude.values);
        cape=cape.assign_coords(latitude=psurf.latitude.values);
        cape=cape.assign_coords(time=psurf.time.values)
        
        cm=plt.cm.Reds
        cm.set_bad(color="grey")
        cm.set_under(color="grey")
        p = cape.sel(mcape_mcin_lcl_lfc='mcape').plot(x="longitude", y="latitude", col="time",\
                         col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},\
                         cmap=cm,vmin=200,vmax=3000,levels=np.arange(200,3000,100),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+init+'_cape.png',  dpi=300, bbox_inches='tight')
        plt.close()

        cape_sel= cape.sel(mcape_mcin_lcl_lfc='mcape',latitude=slice(40,20),longitude=slice(260,290)).values
        cape_sel=cape_sel.reshape(cape_sel.shape[1]*cape_sel.shape[2],cape_sel.shape[0])
        cape_sel[cape_sel<200]=np.nan
        ax = sns.violinplot(cape_sel,color='grey',density_norm='count',common_norm=True)
        ax.set(ylim=(0,3500))
        ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)
        plt.title(model+' CAPE '+ init)
        plt.ylabel('[J kg$^{-1}$]')
        plt.savefig(figpath+model+init+'_capedist.png',  dpi=300, bbox_inches='tight')
        plt.close()

        #cape=cape.sel(time=cape.time.dt.hour.isin([0,12]))
        #srh=srh.sel(time=srh.time.dt.hour.isin([0,12]))
        cm=plt.cm.RdBu_r
        cm.set_bad(color="grey")
        dcape=cape.fillna(0)-ref_cape.fillna(0)
        p = dcape.sel(mcape_mcin_lcl_lfc='mcape').plot(x="longitude", y="latitude", col="time",\
                         col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},\
                         cmap=cm,vmin=-1000,vmax=1000,levels=np.arange(-1000,1010,100),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+init+'_cape_d.png',  dpi=300, bbox_inches='tight')
        plt.close()



['120' '126' '132' '138' '144' '150' '156' '162']
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/fourcastnetv2_2020-04-07T00_to_2020-04-14T00_ldt_168_6.nc


/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/metpy/calc/thermo.py:1396: RuntimeWarning: invalid value encountered in log
  val = np.log(vapor_pressure / mpconsts.nounit.sat_pressure_0c)
/tmp/ipykernel_3765897/3321963432.py:90: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)


['108' '114' '120' '126' '132' '138' '144' '150']
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/fourcastnetv2_2020-04-07T12_to_2020-04-14T12_ldt_168_6.nc


/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/metpy/calc/thermo.py:1396: RuntimeWarning: invalid value encountered in log
  val = np.log(vapor_pressure / mpconsts.nounit.sat_pressure_0c)
/tmp/ipykernel_3765897/3321963432.py:90: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)


['96' '102' '108' '114' '120' '126' '132' '138']
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/fourcastnetv2_2020-04-08T00_to_2020-04-14T12_ldt_156_6.nc


...

/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/metpy/calc/thermo.py:1396: RuntimeWarning: invalid value encountered in log
  val = np.log(vapor_pressure / mpconsts.nounit.sat_pressure_0c)
/tmp/ipykernel_3765897/3321963432.py:90: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)


['0' '6' '12' '18' '24' '30' '36' '42']
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/fourcastnetv2_2020-04-12T00_to_2020-04-14T12_ldt_60_6.nc


/tmp/ipykernel_3765897/3321963432.py:53: UserWarning: Relative humidity >120%, ensure proper units.
  dplevel=metpy.calc.dewpoint_from_relative_humidity(tlevel.values* units('K'), rlevel.values* units.percent)
/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/metpy/calc/thermo.py:1396: RuntimeWarning: invalid value encountered in log
  val = np.log(vapor_pressure / mpconsts.nounit.sat_pressure_0c)
/tmp/ipykernel_3765897/3321963432.py:90: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)


['-12' '-6' '0' '6' '12' '18' '24' '30']
/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/fourcastnetv2_2020-04-12T12_to_2020-04-14T12_ldt_48_6.nc


/tmp/ipykernel_3765897/3321963432.py:53: UserWarning: Relative humidity >120%, ensure proper units.
  dplevel=metpy.calc.dewpoint_from_relative_humidity(tlevel.values* units('K'), rlevel.values* units.percent)
/work/FAC/FGSE/IDYST/tbeucler/default/mfeldman/miniforge3/envs/wb/lib/python3.11/site-packages/metpy/calc/thermo.py:1396: RuntimeWarning: invalid value encountered in log
  val = np.log(vapor_pressure / mpconsts.nounit.sat_pressure_0c)
/tmp/ipykernel_3765897/3321963432.py:90: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(xr_era5.time.dt.strftime('%Y%m%d%H').values,rotation=90)
